# Residuals

Authors: Madeline Keep, Izabella Kornak, Hannah Weinstein


In [115]:
!pip install aguaclara
!pip install molmass

In [116]:
from aguaclara.core.units import unit_registry as u
import aguaclara as ac
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import molmass as mol
!wget https://raw.githubusercontent.com/monroews/4590FSP/master/Fairmont.py
import Fairmont as fmt

--2020-04-27 23:10:29--  https://raw.githubusercontent.com/monroews/4590FSP/master/Fairmont.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2397 (2.3K) [text/plain]
Saving to: ‘Fairmont.py.1’

Fairmont.py.1       100%[===================>]   2.34K  --.-KB/s    in 0s      

2020-04-27 23:10:29 (14.7 MB/s) - ‘Fairmont.py.1’ saved [2397/2397]



# Task Definition

Our goal is to develop a complete analysis of the composition, management, and disposal of residuals from the proposed treatment process.
According to the Fairmont Treatment Plant’s Scope Of Work document, settled solids from the sedimentation process will be mechanically thickened in an on site facility. The preliminary evaluation suggests using equalization basins, rapid mixers, then settling tanks, and introducing coagulants to form and settle out thickened solids. The residual solids will then be further mechanically dewatered at a separate facility, then can be temporarily stored in a sludge hopper (with the capacity to store a minimum of a week’s worth of sludge) before they are transferred via truck to be disposed off-site. The leftover water will be recycled to the head of the Fairmont sedimentation plant. 

A preliminary evaluation has concluded that that centrifuges are most effective active dewatering technology. We will develop a design for the thickening facility and dewatering facility and evaluate their cost and effectiveness. Our thickening facility will take the collected sludge and thicken it, and also even out the flow before the sludge before it enters the dewatering facility. These are expected to be similar to the coagulation and flocculation tanks. The dewatering facility will be composed of centrifuges that will further dewater the sludge to create a sludge cake. There also must be a distribution mechanism for the sludge cake to be moved out of the facility.

We will also develop a more exact estimate of the expected residual output of the process, and determine a minimum capacity for our residual treatment train.


# Executive Summary 

Our main constraints will rely on the amount of sludge that needs to be treated. Those calculations will depend on the treatment specifications of the plant. This will determine the characteristics of the sludge, and allow us to analyze potential solutions with concrete estimations of the costs of management. Our design solution will develop as we calculate these values



# Constraints
*(Relevant regulations, client expectations, site and budget, process capabilities including process design equations, ease of use, operation requirements,maintainence requirements)*

* Amount of material that must be disposed of
* Transportation costs
* Disposal costs
* Discharge requirements/pollution standards
* Residuals management systems may limit overall plant capacity if not designed and operated properly
* Toxicity of residuals
* Specific resistance of sludge
* Amount of water used
  * (“The quantity of waste washwater for both granular and membrane filters will typically comprise from 2 to 5 percent of the total amount of water processed”-MWH)
* Capacity of washwater facility to to accept high, intermittent flows from backwashing 
* Unit filter rn volumes (UFRV)
* Unit backwash volume (UBMV)
* Effective filtration rate and recover or production efficiency of filter
  * (“Typically, waste washwater quantities are 8 m3/m2 (200 gal/ft2). To achieve a filter production efficiency of 95 percent, the UFRV would have to be at least 200 m3/m2 (5000 gal/ft2) a run. At a filtration rate of 12.2 m/h (5 gpm/ft2), a filter run would have to last at least 1000 min between backwash cycles.” -MWH)


# Physics

* At what steps we should treat/recycle residuals
* Typically, waste washwater quantities are 8 m3/m2 (200 gal/ft2). To achieve a filter production efficiency of 95 percent, the UFRV would have to be at least 200 m3/m2 (5000 gal/ft2) a run. At a filtration rate of 12.2 m/h (5 gpm/ft2), a filter run would have to last at least 1000 min between backwash cycles. -MWH
* The TSS/turbidity ratio for most water sources will vary between 1 and 2, with a typical value being about 1.4. For turbidities less than 10 NTU the ratio is nearly equal to 1. -MWH
* Percentage of plant flow that each type of residual contributes to (Table 21-1 MWH)
* Table 21-3 (Physical, chemical, and biological properties used to characterize water treatment plant residuals)
 

Total mass of sludge (adapted fromm equation 21-13 in MWH):
$$M=D_{coagulant}*E_{coagulant}+(Influent \space turbidity-goal\space turbidity)*E_{turbidity}+X$$

$M=$ total sludge produced,  $kg/m^3$

$D_{coagulant}=$ coagulant dose, $kg/m^3$

$E_{coagulant/turbidity}=$ value used to estimate sludge quantities (from table 21-4 in MWH), kg sludge/kg coagulant/turbidity

X= other coagulent aids

from sow: uses ferric chloride and cationic polumer

"The specific gravity of the dry volatile solids is about 1.0 and dry fixed solids about 2.5." (DEU)




In [0]:
def wet_sludge_volume(sg_fixedsolids,sg_volatile,weight_total_dry,percent_solids,percent_volatilematter,temperature):
  #sg fixed solids assumed from MWH 
  rho_water=ac.density_water(temperature)
  sg_solids= 1/(((1-percent_volatilematter)/(sg_fixedsolids)+percent_volatilematter/(sg_volatile)))
  sg_sludge=1/(((1-percent_solids)/(1)+percent_solids/(sg_solids)))
  rho_sludge_wet=sg_sludge*rho_water
  wet_sludge_volume=weight_total_dry/(rho_water*sg_sludge*percent_solids)
  wet_sludge_volume.sg_solids=sg_solids
  wet_sludge_volume.sg_sludge=sg_sludge
  wet_sludge_volume.rho_sludge_wet=rho_sludge_wet
  return wet_sludge_volume


In [130]:
def total_dry_sludge(flow,coag_type='None',coag_dose=0*u.mg/u.L,percent_al=0,\
                     influent_turbidity=0*u.NTU,goal_turbidity=0*u.NTU,removal_ratio=1.4*u.g/(u.m**3*u.NTU),\
                     polymer_dose=0*u.mg/u.L):
  if coag_type =='Alum':
    est_val=.26
  elif coag_type =='Ferric sulfate':
    est_val=.53
  elif coag_type =='Ferric chloride':
    est_val=.66
  elif coag_type =='PACl':
    est_val=.0489*percent_al
  else: est_val=0
  sludge_coag=(flow*coag_dose*est_val).to(u.kg/u.day)
  sludge_turbidity=(flow*(influent_turbidity-goal_turbidity)*removal_ratio).to(u.kg/u.day)
  sludge_polymer=(flow*polymer_dose*1).to(u.kg/u.day)
#HOW DO WE ACCOUNT FOR TOC
#influent_TOC=0*u.mg/u.L,goal_TOC=0*u.mg/u.L
  sludge=sludge_coag+sludge_turbidity+sludge_polymer
  sludge.total=sludge
  sludge.coag=sludge_coag
  sludge.turbidity=sludge_turbidity
  sludge.polymer=sludge_polymer
  return sludge

test=total_dry_sludge(flow=.5*u.m**3/u.s,coag_type='Alum',coag_dose=30*u.mg/u.L,influent_turbidity=25*u.NTU,polymer_dose=1*u.mg/u.L)
print(test)
print(test.coag)
print(test.turbidity)
print(test.polymer)
print(test.turbidity/test)
print(test.polymer/test)
print(test.coag/test)
C_c_in_design = 4 * u.mg/u.L
C_c_out_design = 3 * u.mg/u.L





1892 kilogram / day
337 kilogram / day
1512 kilogram / day
43.2 kilogram / day
0.7991 dimensionless
0.02283 dimensionless
0.1781 dimensionless


In [0]:
#residual production table 21.2
data_residuals={'Min':[(.08),(4),(2),(1.5),(.08),(.3),(2),(10)],'Max':[.3,8,5,10,.3,6,8,50],'Typical':[.1,'-', ('2 or 3'),(5-8),.1,4,6,(20-30)]}
rows_resid=['Alum','Direct filtration backwash water','Filter backwash water','Ion exchange brine','Iron coagulation sludge','Lime-softening sludge','Microfiltration backwash water','Reverse osmosis concentrate']
data_residproduction=pd.DataFrame(data=data_residuals,index=rows_resid)
#physical properties reference table from table 21-5 asumin max values
#alum, iron
data_physproperties={'Min Volume':[.05,.06],'Max Volume':[.15,.15],'Min Total Solids':[.1,.25], 'Max Total solids':[4,3.5],'Min Dry Density':[1200*u.kg/u.m**3,1200*u.kg/u.m**3],'Max Dry Density': [1500*u.kg/u.m**3,1800*u.kg/u.m**3],'Min Wet Density': [1025*u.kg/u.m**3,1050*u.kg/u.m**3], 'Max Wet Density': [1100*u.kg/u.m**3,1200*u.kg/u.m**3],'Min Spec Resistance':[10110*u.m/u.kg,40440*u.m/u.kg],'Max Spec Resistance':[50550*u.m/u.kg,151650*u.m/u.kg]}
rows=['Alum','Iron']
data_chemconstit={'Min Solids(Al/Fe)':[15,4],'Max Solids(Al/Fe)':[40,21],'Min Solids(Silicates and inert mats)':[35,35],'max Solids(Silicates and inert mats)':[70,70],'Min solids(organics/volatile)':[10,5],'Max solids(organics)':[25,15]}
physical_properties=pd.DataFrame(data=data_physproperties,index=rows)
chemical_constituents=pd.DataFrame(data=data_chemconstit,index=rows)

physical_properties

# Table listing potential strategies to meet the dewatering constraints and treatment objectives 


| strategy | advantages  | disadvantages |
|---|---|---|
| Centrifuge | high speed dewatering | high energy input | 
| | 25% and 35% solids by weight.| standby unit needed in some cases |
||works for all sludge types|very loud to run|
||cost less to operate than filter press||
| Plate and Frame Filter Press | stackable, cake-like solids | high potential for blockage and damage | 
| | 40% and 70% solids by weight | potential for leakage |
| |can operate on a continual basis | |
| Belt Filter Press | effective on high-volume waste streams  | difficult maintenance |  |
| | 18% and 25% solids by weight. | suitable for high sludge moisture content|
| | low energy and space taken up| |
||10 year service life||

#Table listing potential strategis to meet sludge handling constraints and objectives

| strategy | advantages  | disadvantages |
|---|---|---|
|Sludge Detention Ponds| can further thicken sludge | 280,000 sq ft of land required|
| | lowers volume of sludge in dumping site| high cost of $46000/day| 
|Sludge Processing Facility| converts hazardous waste into nonhazardous | high cost|
| | has potential to create new materials | still in research phase, not usable|
|Off-Site Dumping | lower cost | hazardous waste dumping | 
| | No extra land or equipment needed | highest volume of waste produced |


# Vendors and contact information file

See this [tab delimited spreadsheet](https://raw.githubusercontent.com/monroews/4590FSP/master/vendors.txt).

Add rows to the vendor spreadsheet as you find them. The sheet can easily be edited in Excel. Just make sure to save it as a tab delimited sheet. Push your changes to github so the entire class has access to the document.



In [0]:
vendors = pd.read_csv('https://raw.githubusercontent.com/monroews/4590FSP/master/vendors.txt',sep='\t')
vendors.head()

# Shared assumptions

See this [python file, Fairmont.py](https://github.com/monroews/4590FSP/blob/master/Fairmont.py)

Use this file to hold parameters that are in the SOW or elsewhere that
influence the design of the entire plant. This provides an easy mechanism
for all of us to share the same assumptions. Read the SOW to find sections that relate to your part of the design and enter all pertinent variables into [Fairmont.py](https://github.com/monroews/4590FSP/blob/master/Fairmont.py) and then push those changes to github.

# Major Decisions

Write several paragraphs describing the major decisions that you will need to make in order to create a design and outline a strategy for how you will obtain the necessary information to make those decisions.

# Next Steps and Challenges

Challenges include trying to reduce the volume of residuals within the plant. This should be done by concentrating wastes as much as possible. This leftover water should be recycled back into the front end of the plant. Also, considering what to do with the residuals can be difficult due to limited space availability of dumping in Los Angeles. Another challenge includes finding safe ways to deal with Arsenic in the residuals, since it is a hazardous waste, and presents itself in high concentrations through the plant inflow. The management of liquid waste streams is also going to be challenging, as our team will try to recycle as much water as possible, and the constituents in the return flow are difficult to manage. 

Next Steps include contacting the other teams to decide which treatment trains we should focus on removing residuals from, and researching more on the types of contaminants in typical inflow waters in the area. This could help our team in determining what types of chemicals and processes will be used in getting rid of contaminants, and the amount of residual flows we will need to focus on. 
We also will need to research the regulatory standards out in place for the contaminants we will be working to remove, in order to understand how and where they can be disposed of. 
Another large part of our next steps includes researching methods of equations for mass of wastes, figure out who manufacturers are of units that process the solids, and suggest centrifuges. reasonable task might be to look at the top options for consolidating solids in addition to centrifuges (other good options) see if you agree if that is the top choice. Identify the "why" of the  technology approach. Helping to think about the overall task, not necessarily this week.


# Design

Begin writing the code to start your design. Define flow rates, mass rates, energy requirements, and any readily calculated dimensions, velocities, flows, etc.

In [0]:
print('The average flow to be treated at Fairmont is',fmt.q_ave.to(u.Mgal/u.day),'or')
print('The maximum flow to be treated at Fairmont is',fmt.q_max.to(u.Mgal/u.day))
specific_energy = (fmt.electricity_power/fmt.q_ave).to(u.J/u.L)
temp = 15 * u.degC
specific_potential_energy = (specific_energy / u.gravity / ac.density_water(temp)).to(u.m)
print('The average amount of energy used per liter of water is expected to be',specific_energy,'or',specific_potential_energy,'of potential energy')

In [0]:


yearly_emulsion_polymer=165*u.ton/u.year
avg_dose_emulsion_polymer=(yearly_emulsion_polymer/(fmt.q_ave)).to(u.mg/u.liter)

dry_sludge_estimate=total_dry_sludge(flow=fmt.q_ave,coag_type='Ferric chloride',coag_dose=fmt.Fe_c_ave,influent_turbidity=fmt.turbidity_c_in_ave,goal_turbidity=fmt.turbidity_c_out_design,polymer_dose=avg_dose_emulsion_polymer)
print('We estimate that the dry sludge produced will be', dry_sludge_estimate.total)




Resources:

https://www.epa.gov/sites/production/files/2015-11/documents/dw-treatment-residuals-mgmt-tech-report-sept-2011.pdf

https://www.epa.gov/sites/production/files/2015-10/documents/method_1684_draft_2001.pdf

http://web.deu.edu.tr/atiksu/ana58/c-7.pdf

#**Questions:**

How do we know what the sludge compostion is?
For physical properties which are we using in the range?

Extra code below:

In [118]:
#defining function for Specifiv gravity of sludge
#eq 21.1
#FIX THIS 
def spec_grav_sludge(weight_total_dry,sg_fixed,sg_volatile,weight_fixed,weight_volatile,temperature):
  #the weights can be inputted as percents where total weight is = 1
  rho=ac.density_water(temperature)
  sg_sludge= weight_total_dry/(rho*(weight_fixed/(rho*sg_fixed)+weight_volatile/(rho*sg_volatile)))
  sg_sludge.temp=temperature
  sg_sludge.rho_water=rho
  return sg_sludge
sg_sludge=spec_grav_sludge(1,2.5,1,.9,.1,25*u.degC)

def density_sludge_wet(sg_sludge):
  rho_sludge=sg_sludge.rho_water*sg_sludge
  return rho_sludge

density=density_sludge_wet(sg_sludge)

def wet_sludge_volume(weight_total_dry,sg_sludge,percent_solids):
  wet_sludge_volume=weight_total_dry/(sg_sludge.rho_water*sg_sludge*percent_solids)
  return wet_sludge_volume
wsv=wet_sludge_volume(1,sg_sludge,.5)
print (wsv)



0.0009227 meter ** 3 / kilogram


In [0]:
#mass and volume fom coag addition 
#inputs are going to be flow rate, raw water turbidity, dose
#sludge solidce concenctraten, solids sg, TSS:turbidity (NTU)
def sludge_coag(coag_type,dose,flow_rate):
  if coag_type =='Alum':
    est_val=.26
  elif coag_type =='Ferric sulfate':
    est_val=.53
  elif coag_type =='Ferric chloride':
    est_val=.66
  #FIND A WAY TO INPUT THIS 
  #elif coag_type =='PACl'
    #est_val=.0489*%Al
  else: est_val=0
  sludge_coag=flow_rate*dose*est_val
  return sludge_coag.to(u.kg/u.day)

sludge=sludge_coag('Alum',30*u.mg/u.L,.5*u.m**3/u.s)
print(sludge)
  
def sludge_turbidity(flow,raw_water_turbidity,removal_ratio=1.4*u.g/(u.m**3*u.NTU)):
  sludge_turbidity=flow*raw_water_turbidity*removal_ratio
  return sludge_turbidity.to(u.kg/u.day)

sludge_t=sludge_turbidity(.5*u.m**3/u.s,25*u.NTU)
print(sludge_t)

def sludge_polymer(flow,polymer_dose):
  sludge_polymer=flow*polymer_dose*1
  return sludge_polymer.to(u.kg/u.day)

sludge_p=sludge_polymer(.5*u.m**3/u.s,1*u.mg/u.L)
print(sludge_p)
